### Webscrape Example with referances to product used

### https://github.com/julia-git/webscraping_ny_mta/blob/master/Webscraping.ipynb
### https://towardsdatascience.com/how-to-web-scrape-with-python-in-4-minutes-bc49186a8460
### https://www.dataquest.io/blog/web-scraping-beautifulsoup/
### https://chrisalbon.com/python/data_wrangling/pandas_regex_to_create_columns/
#### https://medium.com/kenlok/how-to-create-sankey-diagrams-from-dataframes-in-python-e221c1b4d6b0

##### https://www.youtube.com/watch?v=HmQR8Xy9DeM

In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re
import csv
from csv import reader
import pandas as pd
import json
import random

Set the URL you want to webscrape from

In [2]:
url = 'http://catalog.drexel.edu/graduate/collegeofcomputingandinformatics/datascience/#degreerequirementstext'

Connect to the URL

In [3]:
response = requests.get(url)

In [4]:
response #200 means it went through

<Response [200]>

Parse HTML and save to BeautifulSoup object

In [5]:
soup = BeautifulSoup(response.text, "html.parser")

In [6]:
#soup, # remove # if you want to see the output of soup  

Example to locate all 'a' tags

In [7]:
#soup.findAll('a') # remove # if you want to see the output of soup "a" tags

Let's take a quick look at the a tag withthe class of bubbleline code

In [8]:
one_a_tag = soup.find_all('a', class_ = 'bubblelink code')
#print(type(one_a_tag))
#print(len(one_a_tag))
#print(one_a_tag)

We want to extract the actual class title and append it to the list of links

In [9]:
links=[]
class_title = []
for class_title in one_a_tag:
    links.append(class_title['href'])
#print(links)

The full download URL is 'http:/catalog.drexel.edu/' + link

To download the whole data set, let's loop through all a tags that are classes, with using a random timer

In [10]:
one_a_tag = soup.find_all('a', class_ = 'bubblelink code')
course_links = []     

for i in range(0,len(one_a_tag)): #'a' tags are for links
    one_a_tag = soup.find_all('a', class_ = 'bubblelink code')[i]
    links = one_a_tag['href']
    download_url = 'http://catalog.drexel.edu/'+ links
    urllib.request.urlretrieve(download_url)
    time.sleep(random.randint(1,5)) # adding a random sleep timer
    course_links.append(download_url)
#    print(i)
#    print(download_url)

This is a key piece of the puzzel, the full list of potential classes is in hand, converted to a df and csv file.
Next up pull out the classes, descriptions and prerequisites for each of the course offerings

In [11]:
#print(course_links)

df = pd.DataFrame({'course_links':course_links})
#print (df)

df.to_csv('C://Users//Joe//Documents//Drexel_University//Drexel_degree_project//drexeledu_courselist.csv', encoding='utf-8', index=False)

In [12]:
# add the ability to load saved course list
#df = pd.read_csv('C://Users//Joe//Documents//Drexel_University//Drexel_degree_project//drexeledu_courselist.csv')
#display(df)
#course_links = df

In [110]:
df5 = pd.DataFrame()

for i in range(0,len(df)): # length of the potential classes
    course_deatils=[]
    url = course_links[i]
    print(url)
    response = requests.get(url)
    response #200 means it went through
    soup = BeautifulSoup(response.text, "html.parser")
    
    c_h = []
    title = soup.findAll('h2')
    for string in title:
        c_h.append(str(string))
    df1 = pd.DataFrame(c_h, columns = ['Title'])
    df1 = df1.drop(df1.index[1:])
    df1['Course number'] = df1['Title'].str.extract('([A-Z]\w{0,}\s\d\d\d)', expand=True)
    df1['Credits'] = df1['Title'].str.extract('(\d\D\d)', expand=True)
    df1['Description'] = df1['Title'].str.split('([A-Z]\w{0,}\s\d\d\d)').str[2] # creating a sub set of the list
    df1['Title'] = df1['Description'].str.split('(\d\D\d)').str[0]  # keeping the first part to obtain the course title
    df1 = df1.drop(columns=['Description'])
    df1 = df1.dropna()

    c_d = []
    description = soup.findAll('p', class_ = 'courseblockdesc')
    for string in description:
        c_d.append(str(string))
    df2 = pd.DataFrame(c_d, columns = ['Description'])
    df2['Course details'] = df2['Description'].str.split('(\\n)').str[2] # creating a sub set of the list
    df2 = df2.drop(columns=['Description'])
    df2 = df2.dropna()
    
    # taking soup converting to strings and then making a panada data frame
    p_s = []
    prereq = soup.strings
    for string in prereq:
        p_s.append(str(string))
    df3 = pd.DataFrame(p_s, columns =['Prerequisites'])
    df3['place_holder'] = df3.loc[df3['Prerequisites'] == 'Prerequisite:']
    df3 = df3.sort_values(by=['Prerequisites'])
    try:
        df3 = (df3.loc[[int(df3.Prerequisites[df3.Prerequisites == 'Prerequisites:'].index.to_numpy()+1)]])
#        df3 = (df3.loc[[int(df3.Concurrently[df3.Concurrently == '(Can be taken Concurrently)'].index.to_numpy()+1)]])
        df3 = df3.drop(columns=['place_holder'])
        df3 = df3['Prerequisites'].str.findall('([A-Z]\w{0,}\s\d\d\d\s)')
        df3 = pd.DataFrame(df3.values.tolist(), index= df3.index)
        df3 = df3.rename(columns=lambda x: 'Prerequisites_'+(str(x)))
        df3 = df3.reset_index()
        df3 = df3.drop(columns=['index'])
        df3 = df3.dropna(how='all')
    except:
        df3 = []
        data = [['None']]
        df3 = pd.DataFrame(data, columns = ['Prerequisites_0'])    
        df3 = df3.dropna(how='all')
    df4 = pd.DataFrame()
    df4 = pd.concat([df1, df2, df3], axis=1)
    df5 = df5.append(df4, sort=False)

display(df5)

http://catalog.drexel.edu//search/?P=DSCI%20511
http://catalog.drexel.edu//search/?P=DSCI%20521
http://catalog.drexel.edu//search/?P=DSCI%20591
http://catalog.drexel.edu//search/?P=DSCI%20592
http://catalog.drexel.edu//search/?P=CS%20510
http://catalog.drexel.edu//search/?P=CS%20520
http://catalog.drexel.edu//search/?P=CS%20521
http://catalog.drexel.edu//search/?P=CS%20540
http://catalog.drexel.edu//search/?P=CS%20570
http://catalog.drexel.edu//search/?P=CS%20571
http://catalog.drexel.edu//search/?P=CS%20583
http://catalog.drexel.edu//search/?P=CS%20613
http://catalog.drexel.edu//search/?P=CS%20615
http://catalog.drexel.edu//search/?P=CS%20647
http://catalog.drexel.edu//search/?P=CS%20660
http://catalog.drexel.edu//search/?P=CS%20676
http://catalog.drexel.edu//search/?P=INFO%20623
http://catalog.drexel.edu//search/?P=INFO%20624
http://catalog.drexel.edu//search/?P=INFO%20634
http://catalog.drexel.edu//search/?P=INFO%20659
http://catalog.drexel.edu//search/?P=CS%20530
http://catalog.dre

,Title,Course number,Credits,Course details,Prerequisites_0,Prerequisites_1,Prerequisites_2,Prerequisites_3,Prerequisites_4,Prerequisites_5
0,Data Acquisition and Pre-Processing,DSCI 511,3.0,Introduces the breadth of data science through...,None,NaN,NaN,NaN,NaN,NaN
0,Data Analysis and Interpretation,DSCI 521,3.0,Introduces methods for data analysis and their...,None,NaN,NaN,NaN,NaN,NaN
0,Data Science Capstone I,DSCI 591,3.0,Explores data science in practice as an open-e...,None,NaN,NaN,NaN,NaN,NaN
0,Data Science Capstone II,DSCI 592,3.0,Explores data science in practice as an open-e...,DSCI 591,NaN,NaN,NaN,NaN,NaN
0,Introduction to Artificial Intelligence,CS 510,3.0,Well-formed problems; state spaces and search ...,CS 571,CS 520,CS 570,NaN,NaN,NaN
0,Computer Science Foundations,CS 520,3.0,Survey of basic mathematics concepts needed fo...,CS 570,NaN,NaN,NaN,NaN,NaN
0,Data Structures and Algorithms I,CS 521,3.0,Techniques for analyzing algorithms: asymptoti...,CS 571,CS 520,CS 570,NaN,NaN,NaN
0,High Performance Computing,CS 540,3.0,"Covers the design, evaluation and use of high-...",CS 571,CS 520,CS 570,NaN,NaN,NaN
0,Programming Foundations,CS 570,3.0,Develops an understanding of the principles be...,None,NaN,NaN,NaN,NaN,NaN
0,Advanced Programming Techniques,CS 571,3.0,"Covers the Unix operating system, bash shell p...",CS 570,NaN,NaN,NaN,NaN,NaN


In [111]:
df5 = df5.sort_values(by='Course number', ascending=True)
df5 = df5.drop_duplicates(['Course number'])
df5 = df5.reset_index(drop=True)
df5['Prerequisites_0'].replace('None', 'NaN', inplace=True)
df5.to_csv('C://Users//Joe//Documents//Drexel_University//Drexel_degree_project//drexeledu_coursedetails.csv', encoding='utf-8', index=True)
display(df5)


,Title,Course number,Credits,Course details,Prerequisites_0,Prerequisites_1,Prerequisites_2,Prerequisites_3,Prerequisites_4,Prerequisites_5
0,Fundamentals of Databases,CS 500,3.0,This course gives an introduction to data mana...,CS 571,CS 520,CS 570,NaN,NaN,NaN
1,Introduction to Artificial Intelligence,CS 510,3.0,Well-formed problems; state spaces and search ...,CS 571,CS 520,CS 570,NaN,NaN,NaN
2,Computer Science Foundations,CS 520,3.0,Survey of basic mathematics concepts needed fo...,CS 570,NaN,NaN,NaN,NaN,NaN
3,Data Structures and Algorithms I,CS 521,3.0,Techniques for analyzing algorithms: asymptoti...,CS 571,CS 520,CS 570,NaN,NaN,NaN
4,Developing User Interfaces,CS 530,3.0,This course examines the implementation of mul...,CS 571,CS 520,CS 570,NaN,NaN,NaN
5,Interactive Computer Graphics,CS 537,3.0,This is a project-oriented class that covers t...,CS 571,CS 520,CS 570,NaN,NaN,NaN
6,High Performance Computing,CS 540,3.0,"Covers the design, evaluation and use of high-...",CS 571,CS 520,CS 570,NaN,NaN,NaN
7,Programming Foundations,CS 570,3.0,Develops an understanding of the principles be...,NaN,NaN,NaN,NaN,NaN,NaN
8,Advanced Programming Techniques,CS 571,3.0,"Covers the Unix operating system, bash shell p...",CS 570,NaN,NaN,NaN,NaN,NaN
9,Introduction to Computer Vision,CS 583,3.0,Theoretical and algorithmic foundation and app...,CS 571,CS 520,CS 570,NaN,NaN,NaN


In [112]:
# create  copy of just the prerequisities
df6 = df5
df6 = df6.drop(['Title', 'Credits', 'Course details'], axis = 1) 
display(df6)

,Course number,Prerequisites_0,Prerequisites_1,Prerequisites_2,Prerequisites_3,Prerequisites_4,Prerequisites_5
0,CS 500,CS 571,CS 520,CS 570,NaN,NaN,NaN
1,CS 510,CS 571,CS 520,CS 570,NaN,NaN,NaN
2,CS 520,CS 570,NaN,NaN,NaN,NaN,NaN
3,CS 521,CS 571,CS 520,CS 570,NaN,NaN,NaN
4,CS 530,CS 571,CS 520,CS 570,NaN,NaN,NaN
5,CS 537,CS 571,CS 520,CS 570,NaN,NaN,NaN
6,CS 540,CS 571,CS 520,CS 570,NaN,NaN,NaN
7,CS 570,NaN,NaN,NaN,NaN,NaN,NaN
8,CS 571,CS 570,NaN,NaN,NaN,NaN,NaN
9,CS 583,CS 571,CS 520,CS 570,NaN,NaN,NaN


#### Graph theroy on class list - setting up the adjaceny matrix 

In [208]:
# setting up the adjaceny matrix 
lst = 0
#vertices = df6['Course number']

df7 = pd.DataFrame(lst, index = df6['Course number'], columns = df6['Course number'])
display(df7)

Course number,CS 500,CS 510,CS 520,CS 521,CS 530,CS 537,CS 540,CS 570,CS 571,CS 583,...,INFO 633,INFO 634,INFO 648,INFO 659,INFO 662,INFO 690,INFO 691,INFO 712,INFO 725,SE 578
Course number,,,,,,,,,,,,,,,,,,,,,
CS 500,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CS 510,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CS 520,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CS 521,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CS 530,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CS 537,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CS 540,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CS 570,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CS 571,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [215]:
df8 = [[]]
df9 = [[]]

In [217]:
df8 = df7
df9 = df8
columns = list(df6)
columns = columns[1:-1]
for index in range(len(df6['Course number'])):
    print('index', index)
    for column in columns:
        print('column',column)
        if df6.loc[index, column] == 'NaN':
#            print(df6.loc[index, column])
            df8.loc[index, (df6.loc[index, column])] = 0
#            print(df8.loc[index, (df6.loc[index, column])])
        else:
            print(df6.loc[index, column])
            df8.loc[index, (df6.loc[index, column])] = 1
            print(df8.loc[index, (df6.loc[index, column])])
print(df8)

index 0
column Prerequisites_0
CS 571 
1.0
column Prerequisites_1
CS 520 
1.0
column Prerequisites_2
CS 570 
1.0
column Prerequisites_3
nan
1.0
column Prerequisites_4
nan
1.0
index 1
column Prerequisites_0
CS 571 
1.0
column Prerequisites_1
CS 520 
1.0
column Prerequisites_2
CS 570 
1.0
column Prerequisites_3
nan
1.0
column Prerequisites_4
nan
1.0
index 2
column Prerequisites_0
CS 570 
1.0
column Prerequisites_1
nan
1.0
column Prerequisites_2
nan
1.0
column Prerequisites_3
nan
1.0
column Prerequisites_4
nan
1.0
index 3
column Prerequisites_0
CS 571 
1.0
column Prerequisites_1
CS 520 
1.0
column Prerequisites_2
CS 570 
1.0
column Prerequisites_3
nan
1.0
column Prerequisites_4
nan
1.0
index 4
column Prerequisites_0
CS 571 
1.0
column Prerequisites_1
CS 520 
1.0
column Prerequisites_2
CS 570 
1.0
column Prerequisites_3
nan
1.0
column Prerequisites_4
nan
1.0
index 5
column Prerequisites_0
CS 571 
1.0
column Prerequisites_1
CS 520 
1.0
column Prerequisites_2
CS 570 
1.0
column Prerequisites

In [188]:
columns = list(df6)
columns = columns[1:-1]
#print( type(columns))
#print(columns[0])
#print(list(df6))
print(df6.loc[1, 'Prerequisites_0'])

#for column in columns:
#    print(column)
#    print (df6[column][0]) 

CS 571 


In [ ]:
import networkx as nx
#https://networkx.github.io/documentation/stable/tutorial.html#creating-a-graph

In [ ]:
def topological_sort(source):
    """perform topo sort on elements.

    :arg source: list of ``(name, set(names of dependancies))`` pairs
    :returns: list of names, with dependancies listed first
    """
    pending = [(name, set(deps)) for name, deps in source]        
    emitted = []
    while pending:
        next_pending = []
        next_emitted = []
        for entry in pending:
            name, deps = entry
            deps.difference_update(set((name,)), emitted) # <-- pop self from dep, req Py2.6
            if deps:
                next_pending.append(entry)
            else:
                yield name
                emitted.append(name) # <-- not required, but preserves original order
                next_emitted.append(name)
        if not next_emitted:
            raise ValueError("cyclic dependancy detected: %s %r" % (name, (next_pending,)))
        pending = next_pending
        emitted = next_emitted

In [ ]:
# Required Core Courses
# Choose 2


In [ ]:
# A Python program to print all  
# combinations of given length 
from itertools import combinations 

# Get all combinations of [comb] 
# and length 15
comb = combinations(['DSCI 511','DSCI 521','DSCI 591','DSCI 592','CS 510','CS 520','CS 521','CS 540','CS 570','CS 571','CS 583','CS 613',
'CS 615','CS 647','CS 660','CS 676','INFO 623','INFO 624','INFO 634','INFO 659','CS 530','CS 537','CS 630','INFO 608','INFO 633','INFO 648',
'INFO 690','INFO 691','INFO 725','CS 500','CS 590','CS 661','INFO 590','INFO 591','INFO 605','INFO 606',
'INFO 607','INFO 662','INFO 712','SE 578','DSCI 591','DSCI 592'], 15)

# Print the obtained combinations 
for i in list(comb): 
    print i

In [ ]:
course_deatils=[]
url = course_links[43]
print(url)
response = requests.get(url)
response #200 means it went through
soup = BeautifulSoup(response.text, "html.parser")
#soup
#type(soup)

This is to obtain all headers of the h2 type 

In [ ]:
c_h = []
title = soup.findAll('h2')
for string in title:
    c_h.append(str(string))

df1 = pd.DataFrame(c_h, columns = ['Title'])
display(df1)
df1 = df1.drop(df1.index[1:])
display(df1)
df1['Course number'] = df1['Title'].str.extract('([A-Z]\w{0,}\s\d\d\d)', expand=True)
df1['Credits'] = df1['Title'].str.extract('(\d\D\d)', expand=True)
df1['Description'] = df1['Title'].str.split('([A-Z]\w{0,}\s\d\d\d)').str[2] # creating a sub set of the list
df1['Title'] = df1['Description'].str.split('(\d\D\d)').str[0]  # keeping the first part to obtin the course title
df1 = df1.drop(columns=['Description'])
df1 = df1.dropna()

display(df1)

In [ ]:
c_d = []
description = soup.findAll('p', class_ = 'courseblockdesc')
for string in description:
    c_d.append(str(string))
df2 = pd.DataFrame(c_d, columns = ['Description'])
df2['Course details'] = df2['Description'].str.split('(\\n)').str[2] # creating a sub set of the list
df2 = df2.drop(columns=['Description'])
df2 = df2.dropna()

display(df2)

In [ ]:
# taking soup converting to strings and then making a panada data frame
p_s = []
prereq = soup.strings
for string in prereq:
    p_s.append(str(string))
df3 = pd.DataFrame(p_s, columns =['Prerequisites'])
df3['place_holder'] = df3.loc[df3['Prerequisites'] == 'Prerequisite:s']
df3 = df3.sort_values(by=['Prerequisites'])
    try:
        df3 = (df3.loc[[int(df3.Prerequisites[df3.Prerequisites == 'Prerequisites:'].index.to_numpy()+1)]])
        try: 
            df3 = (df3.loc[[int(df3.Concurrently[df3.Concurrently == '(Can be taken Concurrently)'].index.to_numpy()+1)]])
        except:
        df3 = df3.drop(columns=['place_holder'])
        df3 = df3['Prerequisites'].str.findall('([A-Z]\w\s\d\d\d\s)')
        df3 = pd.DataFrame(df3.values.tolist(), index= df3.index)
        display(df3)
        df3 = df3.rename(columns=lambda x: 'Prerequisites_'+(str(x)))
        df3 = df3.reset_index()
        df3 = df3.drop(columns=['index'])
        df3 = df3.dropna()

    except:
        df3 = []
        data = [['None']]
        df3 = pd.DataFrame(data, columns = ['Prerequisites_0']) 
        df3 = df3.dropna()


display(df3)

#df3.to_csv('C://Users//Joe//Documents//drexeledu_coursedetails.csv', encoding='utf-8', index=False)
#type(df3)
#'Prerequisites'

In [ ]:
df4=[]
df4 = pd.concat([df1, df2, df3], axis=1)
display(df4)

In [ ]:
def has_class_but_no_id(tag):
    return tag.has_attr('class') and not tag.has_attr('id')

In [ ]:
soup.find_all(has_class_but_no_id)

In [ ]:
print(soup.find(string=re.compile("^Prerequisites" )))

In [ ]:
print(soup.find(string=re.compile("^Prerequisites" )))

In [ ]:
soup.find_all("p","br")

In [ ]:
soup.findAll('b')

In [ ]:
soup.findAll('b', class_ = 'Prerequisites:')

In [ ]:
print(soup.p.prettify())

In [ ]:
soup.get_text()

In [ ]:
soup.findAll('b')
#print(type(course_deatils))
#print(len(course_deatils))
#print(course_deatils)

In [ ]:
print(df.applymap(lambda text: BeautifulSoup(text, 'html.parser').get_text()))
#soup = BeautifulSoup(response.text, "html.parser")

In [ ]:
div_sub = soup.findAll('div', class_ = 'courseblock')
type(soup.findAll('div', class_ = 'courseblock'))
    
#print(div_sub)

ssplit = string.split()
for word in ssplit:
    print (word)
    
#print(div_sub)    
    

#p = soup.find_all('p')
paragraphs = []
for x in div_sub:
    paragraphs.append(str(x))  #extend

ssplit = string.split()
for paragraphs in ssplit:
    print (paragraphs)
    
print(paragraphs)